# Disk structure

In [1]:
import os
from os import sys
sys.path.append('./newdata')
import yt.units as u
import numpy as np
import matplotlib.pyplot as plt
from species import *
atoms = ['H', 'C', 'O', 'N']
mass = {'H': 1.0079, 'C': 12.0107, 'O': 15.9994, 'N': 14.0067}

All variables set as e.g. krome_idx_H2
plot 'your_output' u 1:krome_idx_H2
 the offset is nkrome= 0


## parameters

use AS 209 data to construct our physical disk model

In [2]:
M_sun = 1.989 * 10**33 * u.gram
Md = 0.028 * M_sun
gamma = 0.4
Rc = 126.0 * u.AU
psi = 0.10
H_100 = 13.3 * u.AU
R_100 = 100.0 * u.AU
h_100 = H_100 / R_100

## density distribution
use following eq.(Andrews et al. 2009) , calculate disk density distribution , meanwhile find number density of H

$$\rho(R,\Theta) = \frac{\Sigma}{\sqrt{2\pi}Rh} \exp{\left[-\frac{1}{2}\left(\frac{\pi/2 - \Theta}{h}\right)^2\right]}$$ 

$$\Sigma = \Sigma_c \left(\frac{R}{R_c} \right)^{-\gamma} \exp{\left[-\left(\frac{R}{R_c} \right)^{2-\gamma}\right]}$$
$$h = h_c \left(\frac{R}{R_c}\right)^{\psi}$$
$$\Sigma_c = (2-\gamma)\frac{M_d}{2\pi R_c^2}$$

In [3]:
def height(radius):
    return (h_100 * (np.abs(radius) / R_100) ** psi)

Sigma_c = ((2 - gamma) * Md / (2 * np.pi * Rc**2))

def suf_mass(radius):
    return (Sigma_c * (Rc / np.abs(radius))**gamma * np.exp(-(np.abs(radius) / Rc)**(2 - gamma)))

def rho(radius, azimuth):
    h = height(radius)
    Sigma = suf_mass(radius)
    return (Sigma / (np.sqrt(2 * np.pi) * np.abs(radius) * h)) * np.exp(-0.5 * ((0.5 * np.pi - azimuth) * u.AU / h)**2)

def get_nH(radius, azimuth):
    return rho(radius, azimuth) / u.mass_hydrogen

get temperature distribution (Jonathan et al. 2014 , Andrews et al. 2009)

$$T = T_c \left(\frac{R}{\text{1AU}}\right)^{-q}$$
$$T_c \approx 200\text{K,  }q \approx 0.55$$

In [4]:
Tc = 200.0 * u.Kelvin

def temp(radius):
    return Tc * (1.0 * u.AU / np.abs(radius))**(0.55)

## column density
integrate number density of H along radiation direction

remember just include contribution from protoplanetary disk , no molecular cloud here

* for face-on disk , radiation will not destroy axial symmetry , thus we can use ($R, \Theta$) 2D model to show disk distribution ; column density should integrate vertically , here we use `get_vertical_ncolH()` to calculate

* for general cases , we need 3D model ; it's necessary to consider a simple way for modelling

* for now , we just calculate points on ONE line (along radiation direction & cross central star) , use `get_ncolH()` to calculate


In [5]:
def column_density(radius, azimuth):
    density = 0 * u.gram / u.cm**2
    r = radius.copy()
    step = np.abs(radius) / 1000.0
    while r <= 900 * u.AU:
        density += rho(r, azimuth) * step
        r += step
        step = np.maximum(np.abs(r) / 1000.0, 0.01 * u.AU)
    return density

def get_ncolH(radius, azimuth):
    density = 0 / u.cm**2
    r = radius.copy()
    step = np.abs(radius) / 1000.0
    while r <= 900 * u.AU:
        density += get_nH(r, azimuth) * step / (np.sin(azimuth))
        r += step
        step = np.maximum(np.abs(r) / 1000.0, 0.01 * u.AU)
    return density

def get_vertical_ncolH(radius, azimuth):
    density = 0 / u.cm**2
    theta = 0.001
    step = 0.001
    while theta < azimuth:
        density += get_nH(radius,theta) * radius * step / ((np.sin(theta))**2)
        theta += step
        #print density
    return density

## chemical evolution
here we just copy from `AGN.ipynb`

* first we should `write_test()` , here we consider only AGN on or off (drop the stop case) , note that you are expected to control $T, n_{\text{H}}, A_V(ncol_{\text{H}})$

* then run `AGN_default()` , here we only consider 4kpc case , and you need to decide the filename for output-data-file (now they are in `./newdata/`)

In [6]:
def write_test(nH, ncolH, temp):
    write_test_Xray(nH, ncolH, temp)
    write_test0(nH, ncolH, temp)

def Av(ncolH):
    return(str(ncolH*5.3e-22)+'d0')

def Tgas(temp):
    return(str(temp)+'d0')

def xH(nH):
    return(str(nH)+'d0')

def write_test_Xray(nH, ncolH, temp):
    with open('./test_Xray.f90', 'w') as f:
        f.write('!###################################################\n\
! WARNING:This is a test auto-generated by KROME, in order to\n\
! show a bare-minimal code to call the KROME\'s subroutine.\n\
! Most of the values could not be appropriate for your\n\
! problem, since this test is only intended as a general\n\
! purpose example.\n\
program test_krome\n\n\
  use krome_main\n\
  use krome_user\n\
  use krome_user_commons\n\
  use krome_ode\n\
  use krome_getphys\n\n\
  implicit none\n\n\
  integer,parameter::nx=krome_nmols\n\
  real*8::x(nx),m(nx+4),Tgas,t,dt,spy,xH,dust2gas,x1(nx)\n\
  integer,parameter::nd=krome_ndust,imax=100\n\
  real*8::xdust(nd),adust(nd),xdusti(nd),data(imax,nd),dataT(imax)\n\
  integer::i,j,k\n\
  spy = 3600. * 24. * 365. !seconds per year\n\
  Tgas = {} !gas temperature (K)\n\
  xH = {} !Hydrogen density\n\n\
  !user commons for opacity and CR rate\n\
  call krome_set_user_av({}) !opacity Av (#)\n\
  call krome_set_user_crate(1.3d-17) !CR rate (1/s)\n\
  call krome_set_user_gas_dust_ratio(1d2) !gas/dust\n\
  call krome_init()\n\n\
  x(:) = 1.d-20\n\
  !initial densities (model EA2 Wakelam+Herbst 2008)\n\
  x(KROME_idx_H2)  = 0.5d0   * xH\n\
  x(KROME_idx_He)  = 9d-2   * xH\n\
  x(KROME_idx_N)   = 7.6d-5  * xH\n\
  x(KROME_idx_O)   = 2.56d-4 * xH\n\
  x(KROME_idx_Cj)  = 1.2d-4  * xH\n\
  x(KROME_idx_Sj)  = 1.5d-5  * xH\n\
  x(KROME_idx_SIj) = 1.7d-6  * xH\n\
  x(KROME_idx_Fej) = 2d-7   * xH\n\
  x(KROME_idx_Naj) = 2d-7   * xH\n\
  x(KROME_idx_Mgj) = 2.4d-6  * xH\n\
  x(KROME_idx_Clj) = 1.8d-7  * xH\n\
  x(KROME_idx_Pj)  = 1.17d-7 * xH\n\
  x(KROME_idx_Fj)  = 1.8d-8  * xH\n\
  x(KROME_idx_GRAIN0) = 1.33d-12 * xH\n\n\
  !calculate elctrons (neutral cloud)\n\
  x(KROME_idx_e) = krome_get_electrons(x(:))\n\n\
  !NOTE: here myCoe array is employed to store the\n\
  ! coefficient values, since the temperature is\n\
  ! constant during the model evolution.\n\
  ! myCoe(:) is defined in krome_user_commons\n\
  !myCoe(:) = krome_get_coef(Tgas,x(:))\n\n\
  dt = 1d2*spy !time-step (s)\n\
  t = 0d0 !initial time (s)\n\n\
  call krome_set_J21xray(1d0)\n\
  !output header\n\
  open(unit=77, file="./newdata/dis")\n\
  write(77,\'(a)\') "#time "//trim(krome_get_names_header())\n\
  x1(:)=x(:)\n\
  m(:)=get_mass()\n\
  k = 0\n\
  do\n\
    print \'(a10,E18.8,a3)\',"time:",t/spy,"yr"\n\
    call krome(x1(:),Tgas,dt) !call KROME\n\
    x1(:)=max(1d-50*xH,x1(:))\n\
    k = k + 1\n\
    t = t + dt !increase time\n\
    !if (mod(k,10) == 0) call jex(nx,t,x1(:),"./data/Trace_inf")\n\
    dt = max(dt,t/10d0) !increase time-step\n\
    write(77,\'(999E15.5)\') t/spy,x1(:)/xH\n\
    if(t>1d8*spy) exit !exit when overshoot 1d8 years\n\
  end do\n\n\
end program test_krome'.format(Tgas(temp), xH(nH), Av(ncolH)))
        
        
def write_test0(nH, ncolH, temp):
    with open('./test0.f90', 'w') as f:
        f.write('!###################################################\n\
! WARNING:This is a test auto-generated by KROME, in order to\n\
! show a bare-minimal code to call the KROME\'s subroutine.\n\
! Most of the values could not be appropriate for your\n\
! problem, since this test is only intended as a general\n\
! purpose example.\n\
program test_krome\n\n\
  use krome_main\n\
  use krome_user\n\
  use krome_user_commons\n\
  use krome_ode\n\
  use krome_getphys\n\n\
  implicit none\n\n\
  integer,parameter::nx=krome_nmols\n\
  real*8::x(nx),m(nx+4),Tgas,t,dt,spy,xH,dust2gas,x1(nx)\n\
  integer,parameter::nd=krome_ndust,imax=100\n\
  real*8::xdust(nd),adust(nd),xdusti(nd),data(imax,nd),dataT(imax)\n\
  integer::i,j,k\n\
  spy = 3600. * 24. * 365. !seconds per year\n\
  Tgas = {} !gas temperature (K)\n\
  xH = {} !Hydrogen density\n\n\
  !user commons for opacity and CR rate\n\
  call krome_set_user_av({}) !opacity Av (#)\n\
  call krome_set_user_crate(1.3d-17) !CR rate (1/s)\n\
  call krome_set_user_gas_dust_ratio(1d2) !gas/dust\n\
  call krome_init()\n\n\
  x(:) = 1.d-20\n\
  !initial densities (model EA2 Wakelam+Herbst 2008)\n\
  x(KROME_idx_H2)  = 0.5d0   * xH\n\
  x(KROME_idx_He)  = 9d-2   * xH\n\
  x(KROME_idx_N)   = 7.6d-5  * xH\n\
  x(KROME_idx_O)   = 2.56d-4 * xH\n\
  x(KROME_idx_Cj)  = 1.2d-4  * xH\n\
  x(KROME_idx_Sj)  = 1.5d-5  * xH\n\
  x(KROME_idx_SIj) = 1.7d-6  * xH\n\
  x(KROME_idx_Fej) = 2d-7   * xH\n\
  x(KROME_idx_Naj) = 2d-7   * xH\n\
  x(KROME_idx_Mgj) = 2.4d-6  * xH\n\
  x(KROME_idx_Clj) = 1.8d-7  * xH\n\
  x(KROME_idx_Pj)  = 1.17d-7 * xH\n\
  x(KROME_idx_Fj)  = 1.8d-8  * xH\n\
  x(KROME_idx_GRAIN0) = 1.33d-12 * xH\n\n\
  !calculate elctrons (neutral cloud)\n\
  x(KROME_idx_e) = krome_get_electrons(x(:))\n\n\
  !NOTE: here myCoe array is employed to store the\n\
  ! coefficient values, since the temperature is\n\
  ! constant during the model evolution.\n\
  ! myCoe(:) is defined in krome_user_commons\n\
  !myCoe(:) = krome_get_coef(Tgas,x(:))\n\n\
  dt = 1d2*spy !time-step (s)\n\
  t = 0d0 !initial time (s)\n\n\
  call krome_set_J21xray(0d0)\n\
  !output header\n\
  open(unit=77, file="./newdata/dis_inf")\n\
  write(77,\'(a)\') "#time "//trim(krome_get_names_header())\n\
  x1(:)=x(:)\n\
  m(:)=get_mass()\n\
  k = 0\n\
  do\n\
    print \'(a10,E18.8,a3)\',"time:",t/spy,"yr"\n\
    call krome(x1(:),Tgas,dt) !call KROME\n\
    x1(:)=max(1d-50*xH,x1(:))\n\
    k = k + 1\n\
    t = t + dt !increase time\n\
    !if (mod(k,10) == 0) call jex(nx,t,x1(:),"./data/Trace_inf")\n\
    dt = max(dt,t/10d0) !increase time-step\n\
    write(77,\'(999E15.5)\') t/spy,x1(:)/xH\n\
    if(t>1d8*spy) exit !exit when overshoot 1d8 years\n\
  end do\n\n\
end program test_krome'.format(Tgas(temp), xH(nH), Av(ncolH)))

In [11]:
# use radius, y/n(for X-ray included or not) to denote our result, only calculate a line
def AGN_default(ncolH, radius, azimuth, label=0, dis=4):
    with open('krome_getphys.f90', 'r') as f:
        lines = f.readlines()
        lines[3713] = 'num2col = max(ncalc,1d-40)*{}\n'.format(str(ncolH/2e4))
    with open('krome_getphys.f90', 'w') as f:
        for l in lines:
            f.write(l)

    nH = './rate_dis/ratexH' + str(dis) + '.dat'
    nHe = './rate_dis/ratexHe' + str(dis) + '.dat'
    orderH = 'cat ' + nH + ' > ratexH.dat'
    os.system(orderH)
    orderHe = 'cat ' + nHe + ' > ratexHe.dat'
    os.system(orderHe)

    os.system('cat ./test_Xray.f90 > ./test.f90')
    os.system('make gfortran')
    os.system('./test')
    command = 'cat ./newdata/dis > ./newdata/{}AU_{:.2f}_y'.format(str(radius), azimuth)
    os.system(command)
    command = 'rm ./newdata/dis'
    #command = 'cat ./data/Trace > ./data/Trace' + str(dis)
    #os.system(command)
    cowsay = 'cowsay -f www I am ionized at ' + str(radius) + ' AU'
    os.system(cowsay)

    '''
    os.system('cat ./test_stop.f90 > ./test.f90')
    os.system('make gfortran')
    os.system('./test')
    command = 'cat ./newdata/dis > ./newdata/{}dis0'.format(str(label)) + str(dis)
    os.system(command)
    cowsay = 'cowsay -f www I am ionized at ' + str(
        dis) + ' kpc, but have been saved at 1E6 YEARS'
    os.system(cowsay)
    '''

    #command = 'rm ./newdata/dis ./newdata/Trace'.format(str(label))
    os.system(command)
    os.system('cat ./test0.f90 > ./test.f90')
    os.system('make gfortran')
    os.system('./test')
    command = 'cat ./newdata/dis_inf > ./newdata/{}AU_{:.2f}_n'.format(str(radius), azimuth)
    os.system(command)
    command = 'rm ./newdata/dis_inf'
    os.system(command)
    os.system('cowsay -f www I feel ok')

## simple test
do rough discretization in range of $\left\{ R \in (200, 300)\text{AU} , \Theta \in (1 , 3)\text{rad} \right\}$

In [12]:
for R in np.arange(200,300,10):
    for theta in np.arange(1.0,3.0,0.5):
        radius = R.copy() * u.AU
        azimuth = theta.copy()
        nH = float(get_nH(radius, azimuth).in_cgs())
        ncolH = float(get_vertical_ncolH(radius, azimuth).in_cgs())
        T = float(temp(radius))
        write_test(nH, ncolH, T)
        AGN_default(ncolH, R, theta)